In [62]:
import pandas as pd
import statistics
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [41]:
data =pd.read_parquet("/content/yellow_tripdata_2022-01.parquet")

In [42]:
data.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [43]:
data['duration'] = data.tpep_dropoff_datetime-data.tpep_pickup_datetime
data.duration = data.duration.apply(lambda td: td.total_seconds()/60)
#data = data[(data.duration >= 1) & (data.duration <= 60)]

In [44]:
data['duration']

0          17.816667
1           8.400000
2           8.966667
3          10.033333
4          37.533333
             ...    
2463926     5.966667
2463927    10.650000
2463928    11.000000
2463929    12.050000
2463930    27.000000
Name: duration, Length: 2463931, dtype: float64

In [45]:
np.sqrt(statistics.variance(data['duration']))

46.44530513776801

In [46]:
statistics.variance(data['duration'])

2157.16636934038

#Outliers

In [53]:
((data.duration >= 1) & (data.duration <= 60)).mean()

0.9827547930522406

In [54]:
data = data[(data.duration >= 1) & (data.duration <= 60)]

# vectorizing

In [55]:
categorical = ['PULocationID', 'DOLocationID']
data[categorical] = data[categorical].astype(str)

In [59]:
train_dicts = data[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [60]:
X_train

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

# Linear Model

In [63]:
target = 'duration'
y_train = data[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190742248472

## validation A to Z

In [68]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [77]:
df_train = read_dataframe('/content/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('/content/yellow_tripdata_2022-02.parquet')

In [78]:
len(df_train), len(df_val)

(2421440, 2918187)

In [79]:
categorical = ['PULocationID','DOLocationID'] 
dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [80]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [81]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.78640662117552